In [1]:
from dotenv import load_dotenv 

load_dotenv() 

True

In [2]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chromadb")

In [3]:
import os
from supabase import create_client, Client

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")
supabase_client = create_client(supabase_url, supabase_key)

## Recommend Article

In [4]:
# Get user preference from database

user_info = supabase_client.table("buddy").select("preferred_topic").eq('buddy_id', 1).execute()
preferred_topics = user_info.model_dump()['data'][0]['preferred_topic']

In [5]:
preferred_topics

['Finance', 'Economy', 'Environment']

In [6]:
# Vector Database

article_collection = chroma_client.get_collection('article')

In [7]:
# Construct Query Sentence

query_sentences = "articles about " + ", ".join(preferred_topics)
query_sentences

'articles about Finance, Economy, Environment'

In [11]:
result = article_collection.query(
            query_texts=query_sentences,
            n_results=10,
        )

result

{'ids': [['15', '3', '8', '10', '11', '13', '7', '9', '12', '1']],
 'distances': [[1.117424887351804,
   1.1906006050282798,
   1.2278199519740098,
   1.252083038949792,
   1.3121058285927636,
   1.3160546740646786,
   1.3556354890954916,
   1.3605243031396395,
   1.3888046486464483,
   1.5320430146703148]],
 'metadatas': [[{'category': 'Finance', 'created_date': '2024-06-05'},
   {'category': 'Finance', 'created_date': '2024-05-25'},
   {'category': 'Finance', 'created_date': '2024-05-29'},
   {'category': 'Finance', 'created_date': '2024-05-31'},
   {'category': 'Finance', 'created_date': '2024-06-01'},
   {'category': 'Finance', 'created_date': '2024-06-03'},
   {'category': 'Finance', 'created_date': '2024-05-28'},
   {'category': 'Finance', 'created_date': '2024-05-30'},
   {'category': 'Finance', 'created_date': '2024-06-02'},
   {'category': 'Productivity', 'created_date': '2024-05-12'}]],
 'embeddings': None,
 'documents': [['The Future of Quantitative Trading in Finance. Explo

## Recommend Mentor

In [18]:
# Vector Database

mentor_collection = chroma_client.get_collection('mentor')

In [19]:
# Will be obtained from user survey

mentor_request = "I want a mentor that works in the Finance department and has experience in Financial Modelling"

In [22]:
result = mentor_collection.query(
            query_texts=mentor_request,
            n_results=1,
        )

result

{'ids': [['1']],
 'distances': [[0.49826343705220727]],
 'metadatas': [[{'department': 'Finance',
    'email': 'john.doe@bank.com',
    'name': 'John Doe',
    'position': 'Financial Analyst',
    'short_desc': 'Analyzes financial data and provides investment guidance.'}]],
 'embeddings': None,
 'documents': [['A mentor from Finance department, working as a Financial Analyst specializing in Investment Analysis. Analyzes financial data and provides investment guidance.']],
 'uris': None,
 'data': None}

## Recommend Buddy

In [23]:
# Vector Database

buddy_collection = chroma_client.get_collection('buddy')

In [26]:
# Get User Description

user = buddy_collection.get("1")
user_description = user['documents'][0]
user_description

'A buddy from Management, with interest in Leadership skills likes Travel, Cooking, Sports. proficient in English with a goal in Networking'

In [28]:
# Get the buddy with the most similar description

result = buddy_collection.query(
        query_texts=user_description, # Use the user_description as the query text
        n_results=1,
        where={
            "$and" : [
                {"years_of_experience" : {"$gte" : user['metadatas'][0]['years_of_experience'] - 2}},      
                {"years_of_experience" : {"$lte" : user['metadatas'][0]['years_of_experience'] + 2}},

                {"interaction_frequency" : user['metadatas'][0]['interaction_frequency']},

                {"meeting_preference" :  user['metadatas'][0]['meeting_preference']},

                {"email" :  {"$ne" : user['metadatas'][0]['email']}},
            ]
        }
    )

result

{'ids': [['9']],
 'distances': [[0.17552574951341257]],
 'metadatas': [[{'buddy_program_goal': 'Providing mentorship',
    'email': 'isabella.foster@ubs.com',
    'hobby': '["Sports", "Cooking"]',
    'instagram': 'isabella_foster_legal',
    'interaction_frequency': 'Weekly',
    'language': '["English"]',
    'linkedin': 'isabella-foster-legal',
    'meeting_preference': 'Combination of both',
    'name': 'Isabella Foster',
    'professional_interest': 'Project management',
    'role': 'Technology',
    'short_desc': 'Provides legal advice and representation.',
    'years_of_experience': 11}]],
 'embeddings': None,
 'documents': [['A buddy from Technology, with interest in Project management likes Sports, Cooking. proficient in English with a goal in Providing mentorship']],
 'uris': None,
 'data': None}